# Final project
## Basics of approximation
### A recursive network model that learns to build sentences in English
The aim of the project is to build and fine-tune a model learning to build correct sentences in English. The collection of books about Harry Potter by J.K. Rowling was used as the teaching text. The neural network will build sentences based on its estimated probability of a given character on the basis of the characters preceding it in the sequence.

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using BSON

┌ Warning: This version of CUDA.jl only supports NVIDIA drivers for CUDA 10.1 or higher (yours is for CUDA 7.5.0)
└ @ CUDA C:\Users\Mateusz\.julia\packages\CUDA\wTQsK\src\initialization.jl:100


In [2]:
text = collect(read("Harry_Potter.txt",String));
alphabet = [unique(text)..., '_'];

### Exploratory analysis
The text on the basis of which the model will learn consists of 7 books from the Harry Potter saga, i.e. nearly 211 thousand. lines of text. The text consists of upper and lower case letters of the English alphabet and special characters. An interesting fact is that there are no numbers in the text, this may be due to the poor recognition of the text in the scanner, which was probably used to digitize the text. You may notice the letter "D" represented as "|)" on one of the first pages. Based on self  memory, there were dates in the text, but due to the volume of the text, it is impossible to fill them in manually.

In [3]:
println(alphabet)

['\ufeff', 'T', 'H', 'E', ' ', 'B', 'O', 'Y', 'W', 'L', 'I', 'V', 'D', '\r', '\n', 'M', 'r', '.', 'a', 'n', 'd', 's', 'u', 'l', 'e', 'y', ',', 'o', 'f', 'm', 'b', 'P', 'i', 'v', 't', 'w', 'p', 'h', 'c', 'k', '’', 'x', 'g', 'j', 'G', ';', '-', 'N', 'A', '“', '”', '—', 'F', 'z', '?', '!', 'q', 'C', 'S', 'R', 'K', '(', ')', ':', 'J', 'U', '"', '‘', 'Z', 'Q', '_']


Onehot coding was used to improve the learning properties of the network

In [4]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

### Architecture and parameters
Sequence length is assumed to be 100 characters because, based on the articles found, the average English sentence is 14.4 words and the average word is 6.47 letters, making an average of 93.17 characters per sentence with no spaces. The project investigated the ability to train the network for various batch_size values which define how many portions of the sequence the network will receive at the entrance to the model.

The model uses the LSTM function, which, due to its architecture, is able to properly filter information
and use them when the source is far away from the current neuron. The network learning ability for batch_size of 32, 64 and 128 was tested on the basis of the value of the loss function. The expected result is that the lattice will learn better (lower loss function value) for lower input batches.

The network consists of 2 hidden LSMT layers and a probability return layer between which the Dropout function was included for regularization.

In [5]:
N = length(alphabet);
seqlen = 100;
batch_size = 32;

In [6]:
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen));

In [7]:
m = Chain(
    LSTM(N, 128),
    Dropout(0.3), 
    LSTM(128, 128),
    Dropout(0.3),
    Dense(128, N),
    softmax)

function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  return l
end

opt = ADAM(0.001)


function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)))
  end
  return String(take!(buf))
end
evalcb = function ()
    @show loss(Xs[5], Ys[5])
    println(sample(m, alphabet, 100))
end

#4 (generic function with 1 method)

In [9]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
bs32 = []
for epoch = 1:10
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        BSON.@save "char_model.bson" m
        best_ls = ls
        last_improvement = epoch
    end
    append!(bs32,ls)
end

┌ Info: Beginning training loop...
└ @ Main In[9]:1
┌ Info: Epoch: 1
└ @ Main In[9]:6


loss(Xs[5], Ys[5]) = 422.07043f0
wdng!zuEowggATOkP’S
)JHw”WOt"(sd)uIYDT—al;kw”OpwzTZR“NFg‘p—t’UFYmnFiVV:l?)?:uIOlw﻿M‘,vJ"ktw"﻿vV QOxs


┌ Info: New best result: 321.77805
└ @ Main In[9]:11
┌ Info: Epoch: 2
└ @ Main In[9]:6


loss(Xs[5], Ys[5]) = 321.6836f0
x.—Kimdv ifayn oron n’litiu y-e“ etsu
Uorvwaf.osw
aos
ul 
fra  Ww,m edo frV tg,tom drHpecm


┌ Info: New best result: 320.03
└ @ Main In[9]:11
┌ Info: Epoch: 3
└ @ Main In[9]:6


...

loss(Xs[5], Ys[5]) = 254.91763f0
zhYronzr m, Heaaf reetisr?” ripitie id Dalasligme 

I” nd hehos.  biofs toavanmo!vwicnct hry re!


┌ Info: New best result: 252.10231
└ @ Main In[9]:11


loss(Xs[5], Ys[5]) = 251.78633f0
yg;nh, gngafatteslcovald titt ang n . phhia row tibd y. awe shyeshehy. Yaas gs se  nis in sit ichih’

┌ Info: Epoch: 10
└ @ Main In[9]:6


┌ Info: New best result: 248.94778
└ @ Main In[9]:11


In [10]:
N = length(alphabet);
seqlen = 100;
batch_size = 64;
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen));

In [11]:
m = Chain(
    LSTM(N, 256),
    Dropout(0.3), 
    LSTM(256, 256),
    Dropout(0.3),
    Dense(256, N),
    softmax)

#9 (generic function with 1 method)

In [12]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
bs64 = []
for epoch = 1:10
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        BSON.@save "char_model.bson" m
        best_ls = ls
        last_improvement = epoch
    end
    append!(bs64,ls)
end

┌ Info: Beginning training loop...
└ @ Main In[12]:1
┌ Info: Epoch: 1
└ @ Main In[12]:6


loss(Xs[5], Ys[5]) = 411.23767f0
r﻿B—UyaApqfoId!C_?rvb"V“jKEE﻿
)_IL“K﻿aZ"(SO)ODF-jo_xFa’C‘n—
Qa—EpQptwti;r’:GtJOns.﻿LjYB’Q”— dnU_Z)zO


┌ Info: New best result: 320.70786
└ @ Main In[12]:11
┌ Info: Epoch: 2
└ @ Main In[12]:6


loss(Xs[5], Ys[5]) = 320.67307f0
. l.t;r’oa ucl a
 lgtemeuaerido ftrmsrmirHstitirgT stot
aas”
nu dwe rer


┌ Info: New best result: 319.2849
└ @ Main In[12]:11
┌ Info: Epoch: 3
└ @ Main In[12]:6


...

┌ Info: New best result: 260.25836
└ @ Main In[12]:11
┌ Info: Epoch: 9
└ @ Main In[12]:6


loss(Xs[5], Ys[5]) = 260.05893f0
yokUhkaot’ 
he cid tutooe thaf d car relkt arecpofdo oceoih os cauorheat an a itnsocheolns woiait n


┌ Info: New best result: 258.0029
└ @ Main In[12]:11
┌ Info: Epoch: 10
└ @ Main In[12]:6


loss(Xs[5], Ys[5]) = 257.88885f0
JEvqpflumbiy itt, 

 wle sir ahe 
HyT ust,’ sapuy sre tly b 
I
bh
Hh
s, srd 
foHoleso


n 
loss(Xs[5], Ys[5]) = 256.7641f0
(meWweiagtsseslereosasd B sceHad troya
 sintofl o reomeicat d bitrigm efr o . wiseu
hlirk meofgror


┌ Info: New best result: 255.86424
└ @ Main In[12]:11


In [23]:
N = length(alphabet);
seqlen = 100;
batch_size = 128;
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen));

In [24]:
m = Chain(
    LSTM(N, 512),
    Dropout(0.3), 
    LSTM(512, 512),
    Dropout(0.3),
    Dense(512, N),
    softmax)

#14 (generic function with 1 method)

In [25]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
bs128 = []
for epoch = 1:10
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        BSON.@save "char_model.bson" m
        best_ls = ls
        last_improvement = epoch
    end
    append!(bs128,ls)
end

┌ Info: Beginning training loop...
└ @ Main In[25]:1
┌ Info: Epoch: 1
└ @ Main In[25]:6


loss(Xs[5], Ys[5]) = 398.93127f0
sSNBO
ce
KAcydPaVL?"?:gBzvJ,’NDih RaqIjPg 
dgjohd xR"x,DvY“Mf’Vfed:WU?F—toG"jq()-toVg_i
loss(Xs[5], Ys[5]) = 320.28763f0
lyw?ro—oagi t    ’rorxdoodadoesnireish.nso r s g yb oe
hecr iec— adnryrha 


┌ Info: New best result: 320.21646
└ @ Main In[25]:11
┌ Info: Epoch: 2
└ @ Main In[25]:6


loss(Xs[5], Ys[5]) = 320.27628f0
E(!Pynfhdi) ewI
i    
nrn”aisGsfoasane nDgeKr,td dleophyim” ee HH stMedgh trhnuuoa
oolvkshnrawsr
loss(Xs[5], Ys[5]) = 319.4048f0
GRz”
h fsellPte   d
wmrreae sr p
r 
eqGt.nseott u sp.g  i 
hsumDd wwoanfon
trefes.e


┌ Info: New best result: 319.54654
└ @ Main In[25]:11
┌ Info: Epoch: 3
└ @ Main In[25]:6


...

┌ Info: Epoch: 9
└ @ Main In[25]:6


ECwH

c
nTovm e iharit atsor cf, t e Pg “o t’hr’ lsh d eig
loss(Xs[5], Ys[5]) = 277.3621f0
cinnsrs ew, rih. poH I. e Dals peeuigdbssorh ondr
nt wi—,s” ulerTteid  


┌ Info: New best result: 275.6532
└ @ Main In[25]:11
┌ Info: Epoch: 10
└ @ Main In[25]:6


loss(Xs[5], Ys[5]) = 276.12375f0
ud  t ulkAHkrbn hy” nitc htrRan ..” 
“tecanenS wty  fsis hanero
nho biiY 
loss(Xs[5], Ys[5]) = 272.46347f0
_Aabh
hhohtot hemvt eop w ome und ,aahe r, v cutd.,.
rg  llddr. auit n n cn tisdrsrvus


┌ Info: New best result: 268.07843
└ @ Main In[25]:11


In [1]:
using Plots
plot(bs32,label = "Batch_size 32")
plot!(bs64, label = "Batch_size 64")
plot!(bs128, label = "Batch_size 128")

SyntaxError: invalid syntax (2537353997.py, line 1)

### Summary
As expected, the best result was achieved by the network with batch_size = 32, which is the smallest examined. The size of the set in relation to the computer's performance turned out to be a big problem when learning the network on this set. In the project, the network was set to 10 iterations to reduce computation time, and conclusions were drawn on this basis. In this number of iterations, the network has learned roughly the word length of the English language. In the first attempt, the network was set to 50 iterations, but the learning process lasted nearly 40 hours, after which the computer refused to cooperate further. The data resulting from the first attempt was lost, but the network already around 40 iterations correctly spelled the name of the main character in most cases, which is quite a surprise. This is probably due to the fact that "Harry" is the word most often found in the pages of the saga.

### Bibilography
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/
* https://fluxml.ai/Flux.jl/stable/
* Length-frequency statistics for written English, G.A.MillerE.B.NewmanE.A.Friedman, Harvard University 1958
* https://medium.com/@theacropolitan/sentence-length-has-declined-75-in-the-past-500-years-2e40f80f589f